# Foundations of Machine Learning - Session 04

- *Course*: Foundations of Machine Learning
- *Session*: 04
- *Unit*: Bayes Classifier

This notebook develops a Naive Bayes classifier in Numpy.

In [1]:
import numpy as np

## Data

For this notebooks' classifiction task, we will utilize text data for the first time. The code block below is pre-filled to load the Twenty Newsgroups dataset, which comprises newsgroups posts on 20 topics. One is displayed below for reference.

To make the text data machine-readable, we transform each post into a vector of token counts. The dataset is thus transformed into a matrix of size `(n_samples, n_terms)` where `n_samples` is the number of posts and `n_terms` is the size of the vocabulary. Each row of the matrix is a feature vector, and each cell $(i,j)$ denotes how often term $j$ occurs in document $i$.

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

data = fetch_20newsgroups(remove=["headers", "footers", "quotes"])

# CountVectorizer represents each text as vector of token counts
vectorizer = CountVectorizer(max_features=2**15)

# Assemble D
D = list(zip(
	vectorizer.fit_transform(data.data).toarray(),
	data.target
))

print("-"*10, data.target_names[data.target[0]], "-"*10)
print(data.data[0])

---------- rec.autos ----------
I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


Another new concept compared to the last week is train/test splits. To make sure that the classifier generalizes well, we want to test its performance on data it has never encountered before. Therefore, we reserve some data for testing, and only use a subset for training.

**Exercise**: split $D$ into two random subsets, a training set $D_\text{train}$ which contains 80% of the data and a test set $D_\text{test}$ which contains 20% of the data.

In [3]:
# Shuffle D
np.random.shuffle(D)

# 80:20 split into train and test set
D_train = D[:int(len(D) * 0.8)]
D_test = D[int(len(D) * 0.8):]

## Naive Bayes Classification

Recall the definition of the naive bayes classifier from lecture slide [ML:VII-84]():
$$
\begin{aligned}
P(A_i \mid B_1,...,B_p) &= \frac{P(A_i) \cdot P(B_1, ..., B_p \mid A_i)}{P(B_1, ..., B_p)} \\
						&= \frac{P(A_i)\cdot\prod^{p}_{j=1} P(B_j  \mid A_i)}{\sum^{k}_{i=1}P(A_i)\cdot\prod^{p}_{j=1} P(B_j\mid A_i)}
\end{aligned}
$$

Here, $P(A_i)$ is the prior probability of class $A_i$, and $P(B_j\mid A_i)$ is the posterior probability of feature $B_j$ given $A_i$. To build a bayes classifier, we first have to infer both of these probabilities from the training data.

## Estimating Prior Probabilities

**Exercise**: compute the prior probability for every class in the dataset.

In [4]:
def priors(D):
	"""
	Computes the prior probabilities P(A_i) from D.

	:param D:  training dataset to estimate prior probabilities from
	:return:  one-dimensional array of size (n_classes) where each cell denotes the prior probability P(A_i) of class A_i
	"""
	class_names, counts  = np.unique(np.array([x[1] for x in D]), return_counts=True)
	return np.array(counts / counts.sum())

priors(D_train)

array([0.04386256, 0.05137554, 0.05270136, 0.05380621, 0.05159651,
       0.05292233, 0.05225942, 0.05281184, 0.05259087, 0.0523699 ,
       0.05192796, 0.05126505, 0.05270136, 0.05281184, 0.05248039,
       0.05170699, 0.04706662, 0.04960778, 0.04143189, 0.03270357])

## Estimating Posterior Probabilities

**Exercise**: compute the posterior probabilities for every feature/class combination in the dataset.

In [5]:
def posteriors(D):
	"""
	Computes the posterior probabilities P(A_i | B_j) for all classes A_i and all features B_j

	:param D: Dataset to estimate probabilities from
	:return: two-dimensional array of size (n_classes, n_features) where cell (i, j) denotes P(A_i | B_j)
	"""
	# Infer Classes from D
	classes = np.array([x[1] for x in D])
	# Assemble feature matrix from D for more efficient lookup
	feature_matrix = np.array([x[0] for x in D])

	# Number of classes
	n_classes = np.unique(classes).shape[0]
	# Number of features
	n_features = feature_matrix.shape[1]

	# Empty array to hold posterior probabilities
	posterior_prob = np.zeros(shape=(n_classes, n_features))

	for i in range(n_classes):
		posterior_prob[i] = feature_matrix[classes == i].sum(axis=0)

	return posterior_prob / posterior_prob.sum(axis=1).reshape(-1, 1)

posteriors(D_train)

array([[0.00000000e+00, 2.02133957e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.36374156e-04, 1.34549662e-04, 0.00000000e+00, ...,
        0.00000000e+00, 2.69099325e-05, 0.00000000e+00],
       [9.77605843e-05, 2.09486966e-05, 0.00000000e+00, ...,
        7.68118877e-05, 0.00000000e+00, 1.25692180e-04],
       ...,
       [3.85388632e-05, 7.38661545e-04, 6.42314387e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.32119096e-04, 4.62416836e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.63810897e-05, 1.14667628e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

## Assembling the classifier

Given both prior and posterior probabilities, we can implement the bayes classifier using the formula given previously. However, one problem might occur: if a term never occurs in a class ($P(B_j | A_i) = 0$, no matter what other features occur in a sample, the class prediction will always be 0. Therefore, we add a small value $\alpha$ to every probability to avoid multiplying by 0.

**Exercise**: implement a Bayes classifier following the formula from the lecture.

*Hint*: add $\alpha$ to the prior and posterior probabilities to increase robustness.

In [6]:
def predict(D, priors, posteriors, alpha=1e-6, return_probs=False):
	"""
	Predicts classes for test dataset D.

	:param D: test dataset to predict classes for
	:param priors: prior probabilities from the train dataset
	:param posteriors: posterior probabilities from the test dataset
	:param alpha: alpha-value to add to the probability inputs
	:param return_probs: whether to return raw probabilities (True) or just the most likely class label (False)
	:return: if returning raw probabilities, returns array of size (n_samples, n_classes) where each cell denotes the class probability for that sample; if returning class labels, returns array of size (n_samples), where each cell is the most likely class of that sample
	"""
	n_classes = posteriors.shape[0]
	n_samples = len(D)
	predictions = np.zeros(shape=(n_samples, n_classes))
	# Adding alpha to deal with terms that never occur for a class
	# Otherwise, documents containing that term could never be assigned that class (product reduces to 0)!
	priors = priors + alpha
	posteriors = posteriors + alpha

	for i, (x, c) in enumerate(D):
		term_mask = x.astype(bool)
		predictions[i] = np.prod(posteriors[:, term_mask] + alpha, axis=1) * priors
		prob_sum = predictions[i].sum()
		predictions *= prob_sum

	if return_probs:
		return predictions
	else:
		return np.argmax(predictions, axis=1)


predict(D_test, priors(D_train), posteriors(D_train), return_probs=True)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Investigate the output of the classifier for $D_\text{test}$.

## Rebuilding the classifier in log space

In the previous step, we can observe that almost every prediction equals 0! This is due to our naive implementation: since we multiply many probabilities, the value eventually underflows, i.e. is smaller than the smallest float64-representable number. Thus, there is no discernible difference to 0 anymore.

The solution is implement our classifier using [log probabilities](https://en.wikipedia.org/wiki/Log_probability) instead. Simply transform the probabilities to log space and replace mathematical operations in your implementation using the log-space equivalents. For convenience, `log_add` is implemented below. If you are new to log probabilities, give the linked resources a read.

In [7]:
def log_add(a, b):
	# Addition in log space, see https://en.wikipedia.org/wiki/Log_probability#Addition_in_log_space for derivation of formula
	return a + np.log1p(np.exp(b-a))

**Exercise**: adapt your `predict` implementation to log-space.

*Hint*:
- `reduce` might come in handy when constructing sums of log-probability arrays.
- do not forget to cast back the final predictions to real space using `np.exp`

In [8]:
from functools import reduce

def predict_log(X, priors, posteriors, alpha=1e-6):
	"""
	Predicts classes for test dataset D in log space.

	:param D: test dataset to predict classes for
	:param priors: prior probabilities from the train dataset
	:param posteriors: posterior probabilities from the test dataset
	:param alpha: alpha-value to add to the probability inputs
	:param return_probs: whether to return raw probabilities (True) or just the most likely class label (False)
	:return: if returning raw probabilities, returns array of size (n_samples, n_classes) where each cell denotes the class probability for that sample; if returning class labels, returns array of size (n_samples), where each cell is the most likely class of that sample
	"""
	n_classes = posteriors.shape[0]
	n_samples = X.shape[0]
	predictions = np.zeros(shape=(n_samples, n_classes))

	# Cast priors and posteriors to log space to avoid underflow
	# Adding alpha to deal with terms that never occur for a class
	priors = np.log(priors + alpha)
	posteriors = np.log(posteriors + alpha)

	for i, x in enumerate(X):
		# Construct term mask to only deal with observed terms
		term_mask = x.astype(bool)
		# Sum instead of multiplying since we are in log space!
		predictions[i] = np.sum(posteriors[:, term_mask], axis=1) + priors
		# log_add instead of + since we are in log space!
		log_prob_sum = reduce(log_add, predictions[i, :])
		# Minus instead of / since we are in log space!
		predictions[i] -= log_prob_sum
	# Cast back to real space and return
	return np.exp(predictions)


p = predict_log(np.array([x[0] for x in D_test]), priors(D_train), posteriors(D_train))

/var/folders/54/8lplw84d1sbgcm1wmd2rgyd40000gn/T/ipykernel_69087/1836505824.py:3: RuntimeWarning: overflow encountered in exp
  return a + np.log1p(np.exp(b-a))


## Evaluation

**Exercise**: evaluate your classifier by comparing the predicted labels to the ground-truth labels of $D_\text{test}$.

*Hint*: you can either use your own evaluation implementation from a previous lab, or use the `sklearn` metric implementations.

In [9]:
from sklearn.metrics import classification_report

print(classification_report(p.argmax(axis=1), [d[1] for d in D_test]))

              precision    recall  f1-score   support

           0       0.71      0.51      0.59       116
           1       0.67      0.58      0.62       139
           2       0.01      1.00      0.02         1
           3       0.78      0.36      0.49       222
           4       0.81      0.69      0.74       131
           5       0.75      0.73      0.74       117
           6       0.72      0.79      0.76       102
           7       0.75      0.67      0.71       130
           8       0.74      0.73      0.73       123
           9       0.82      0.91      0.86       111
          10       0.81      0.96      0.88       109
          11       0.70      0.85      0.77       108
          12       0.64      0.72      0.68       102
          13       0.85      0.90      0.88       110
          14       0.71      0.82      0.76       103
          15       0.79      0.73      0.76       143
          16       0.78      0.76      0.77       123
          17       0.83    